Créer un entier aléatoire pouvant servir
de seed à un wallet de façon sécurisée

In [ ]:
import secrets
rand_numb=secrets.randbits(128)

In [ ]:
rand_numb

26015536743426440181954343574826827649

In [ ]:
bin(rand_numb)

'0b10011100100100110100011111101110101111111101101001000000101000111100111011100001011110110100000001001111011101011111110000001'

In [ ]:
hex_rand_num = hex(rand_numb)
private_key = hex_rand_num[2:]
private_key

'139268fdd7fb481479dc2f6809eebf81'

In [ ]:
import hashlib
m = hashlib.sha256()
m.update(bytes.fromhex(private_key))
res = m.digest()
res

b'\xb3B\xd7\xa4\xbb!\xfb?\xc8\x8f\x7f*2\xe7\x7f\x1dv\xf6_\xb0\xcdd\xa2B.\x11\x80I\xcd\xe6\xd9h'

In [ ]:
hexRes = res.hex()
hexRes

'b342d7a4bb21fb3fc88f7f2a32e77f1d76f65fb0cd64a2422e118049cde6d968'

In [ ]:
checksum = hexRes[0]
checksum

'b'

In [ ]:
full_private_key = str(private_key) + checksum
full_private_key

'139268fdd7fb481479dc2f6809eebf81b'

Représenter cette seed en binaire et le
découper en lot de 11 bits

In [ ]:
binKey = bin(int(full_private_key, 16))[2:].zfill(132)
binKey

'000100111001001001101000111111011101011111111011010010000001010001111001110111000010111101101000000010011110111010111111100000011011'

In [ ]:
def splitBinKey(binKey):
  tab = []
  start = 0
  stop = 11

  for i in range(0,12):
    tab.append(binKey[start:stop])
    start += 11
    stop += 11

  return tab


def splitBinKey2(new_key):
  subList = [new_key[n:n+11] for n in range(0, len(new_key), 11)]

  return subList


splitKey = splitBinKey(binKey)
splitKey

['00010011100',
 '10010011010',
 '00111111011',
 '10101111111',
 '10110100100',
 '00001010001',
 '11100111011',
 '10000101111',
 '01101000000',
 '01001111011',
 '10101111111',
 '00000011011']

Attribuer à chaque lot un mot selon la
liste BIP 39 et afficher la seed en
mnémonique

In [ ]:
def convertToDecimal(binTab):
  intTab = []

  for i in range(0, len(binTab)):
    b = splitKey[i] # binary
    h = f'{int(b, 2):X}' # convert to hex
    i = int(h, 16) # convert to int
    intTab.append(i) # add to the table

  return intTab

intKey = convertToDecimal(splitKey)
intKey

[156, 1178, 507, 1407, 1444, 81, 1851, 1071, 832, 635, 1407, 27]

In [ ]:
with open('seedWords.txt',"r",encoding="utf-8") as f:
  word_list = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
  word_list = [x.strip() for x in word_list] 

In [ ]:
len(word_list)

2048

In [ ]:
mnemonic = []
for itm in intKey:
  mnemonic.append(word_list[itm])

mnemonic

['asticot',
 'machine',
 'décembre',
 'oxyde',
 'payer',
 'alliage',
 'tamiser',
 'jaillir',
 'fissure',
 'égarer',
 'oxyde',
 'acerbe']

In [ ]:
seedPhrase = ''

for elem in mnemonic:
  seedPhrase += elem + ' '

seedPhrase = seedPhrase[:-1] # suppression de l'espace en trop

seedPhrase

'asticot machine décembre oxyde payer alliage tamiser jaillir fissure égarer oxyde acerbe'

Permettre l’import d’une seed
mnémonique

In [ ]:
# Executez la cellule pour importer votre seed

# word_list : liste des 2048 mots
def mnemonique_to_dec(word_list):
  imported_seed = input('Collez votre seed mnémonique ici: ')
  imported_seed = imported_seed.split(' ')
  decimal_list=[]
  for itm in imported_seed:
    for i in range(0,len(word_list)):
      if itm == word_list[i]:
        decimal_list.append(i)
  return decimal_list

decList = mnemonique_to_dec(word_list)

print(decList)

def decToBinList(dec_list):
  binL = []

  for elem in dec_list:
    binL.append(bin(int(elem))[2:])

  return binL

binList = decToBinList(decList)

print(binList)

def completeBin(bin_list):
  new_list=[]
  for itm in bin_list:
    res =''
    i=11-len(itm)
    for j in range(0,i):
      res+='0'
    res+=itm
    new_list.append(res) 
  return new_list

complete_binList = completeBin(binList)

print(complete_binList)

Collez votre seed mnémonique ici: asticot machine décembre oxyde payer alliage tamiser jaillir fissure égarer oxyde acerbe
[156, 1178, 507, 1407, 1444, 81, 1851, 1071, 832, 635, 1407, 27]
['10011100', '10010011010', '111111011', '10101111111', '10110100100', '1010001', '11100111011', '10000101111', '1101000000', '1001111011', '10101111111', '11011']
['00010011100', '10010011010', '00111111011', '10101111111', '10110100100', '00001010001', '11100111011', '10000101111', '01101000000', '01001111011', '10101111111', '00000011011']


Extraction private key et chain code

In [ ]:
mnemonic = seedPhrase
salt = "mnemonic"
iterations = 2048
keyLength = 64

seed = hashlib.pbkdf2_hmac('sha512', mnemonic.encode(), salt.encode(), iterations, keyLength)

hexSeed = seed.hex()
hexSeed

'659490230bc1be3d7a6ea0dc865b0af96ebaefca9b9901a669be425398c1f754ab82c657d6f158f08f8dcdf79f39002d6632c63d3d7f49d28b2617ae10f07610'

In [ ]:
master_privateKey = hexSeed[:keyLength]
chainCode = hexSeed[keyLength:]

print(master_privateKey)
print(chainCode)

659490230bc1be3d7a6ea0dc865b0af96ebaefca9b9901a669be425398c1f754
ab82c657d6f158f08f8dcdf79f39002d6632c63d3d7f49d28b2617ae10f07610


Extraction de la master public key


In [ ]:
pip install ecdsa

In [ ]:
import ecdsa.util

In [ ]:
def get_point_pubkey(point):
 if point.y() & 1:
  key = '03' + '%064x' % point.x()
 else:
  key = '02' + '%064x' % point.x()
 return key

In [ ]:
def getPublicKey(masterPrivateKey):
  _p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
  _r = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141
  _b = 0x0000000000000000000000000000000000000000000000000000000000000007
  _a = 0x0000000000000000000000000000000000000000000000000000000000000000
  _Gx = 0x79BE667EF9DCBBAC55A06295CE870B07029BFCDB2DCE28D959F2815B16F81798
  _Gy = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8

  curve_secp256k1 = ecdsa.ellipticcurve.CurveFp(_p, _a, _b)
  generator = ecdsa.ellipticcurve.Point(curve_secp256k1, _Gx, _Gy, _r)
  point = generator * int(masterPrivateKey, 16)
  
  return get_point_pubkey(point)

In [ ]:
master_publicKey = getPublicKey(master_privateKey)
master_publicKey

'03bf3b6417d0ab32e3cd373d4a6b916247d02e8fea2515fe77f20fbe8f3d30187a'

Générer une clé enfant

In [ ]:
import hmac

def genererCleEnfant(index=0):
  orderOfCurve = 115792089237316195423570985008687907852837564279074904382605163141518161494337 # ordre de la courbe connu

  data = master_publicKey + str(index)
  key = chainCode

  ckd = hmac.new(bytes.fromhex(key), data.encode('utf-8'), hashlib.sha512).hexdigest()

  newChainCode = ckd[64:]

  childPrivateKey = int(ckd[:64] + master_publicKey, 16) % orderOfCurve
  childPrivateKey = hex(childPrivateKey)[2:]

  return newChainCode, childPrivateKey

childChainCode, childPrivateKey = genererCleEnfant()

print('child ChainCode :', childChainCode)
print('child Private Key :', childPrivateKey)
print('child Public Key :', getPublicKey(childPrivateKey))

child ChainCode : 41c489ea70f6fad72184e698faf7e8749c8b66d22179f2b26d05e4acdb003c55
child Private Key : 910c30f1821249346de6085201ed1a809607ad71dbccbaf716f1ad4ece47867b
child Public Key : 022de8f5558b17221339ea17e97aa0206d43c4d7ab7895f70fa49c7952cec1835d


Génerer une clé enfant à partir d'un indice N


In [ ]:
i = input('Veuillez entrer l\'indice de générationde la clé enfant (entre 0 et 2147483647)')

childChainCode, childPrivateKey = genererCleEnfant(i)
print('Génération d\'une clé enfant à l\'indice', i)
print('child ChainCode :', childChainCode)
print('child Private Key :', childPrivateKey)
print('child Public Key :', getPublicKey(childPrivateKey))

Veuillez entrer l'indice de générationde la clé enfant (entre 0 et 2147483647)8
Génération d'une clé enfant à l'indice 8
child ChainCode : 4513a1de852004f46ce072fcbfba370a42cc345d7f99ec4937880f878cabf50b
child Private Key : 4965b9e80616069e07f99e9dff7b08d579f54f5a9262d755d8da818a92606601
child Public Key : 02e03a74315b1ceae82f810c2744def6a846d804e5897c5df95c26a5bc97fc18ff


# Générer un enfant à l'index N à la dérication M avec l'aide de la librairie bip32.py

In [ ]:
!pip install bip32
from bip32 import BIP32, HARDENED_INDEX
bip32seed = BIP32.from_seed(bytes.fromhex(hexSeed))

     |████████████████████████████████| 522kB 7.2MB/s 
     |████████████████████████████████| 112kB 18.3MB/s 


In [ ]:
N = input('Index N: ')
M = input('Dérication M: ')

pubKey = bip32seed.get_extended_pubkey_from_path("m/"+M+"/"+N)

print('Public Key of '+N+' child at derivation '+M+' : '+str(pubKey[1].hex()))

privKey = bip32seed.get_extended_privkey_from_path("m/"+M+"/"+N)

print('Private Key of '+N+' child at derivation '+M+' : '+str(privKey[1].hex()))

Index N: 5
Dérication M: 3
Public Key of 5 child at derivation 3 : 0207c09b1d26a0f2229b65fae4b3c4b0629db7f978753bd3cede05b0a7b97c2ef0
Private Key of 5 child at derivation 3 : f8f3ab15ddd5cb2528e55fadd3750514ac614cc4e5d7fe9e92c49166954828d8
